## Cargar modulos

In [1]:
import os
import sys

import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

Crear sesión de spark

In [2]:
spark : SparkSession= SparkSession.builder.appName("Mi primera app").getOrCreate()

In [3]:
spark

## Datos

Decidí utilizar estos datos por diversos motivos:
* Los datos son de dominio público y en la plataforma de kaggle tiene una calficación de 10 en usabilidad.
* Tiene una buena cantidad de columnas y registros.
* Pero debido a las capacidades de mi computuradora solo analizaré los datos del 2021,
    los cuales cuenta con 6,311,871 filas y 61 columnas. Pero cuenta con columnas redundantes que se eliminaran, en total se elegieron 28 columnas.
* Los datos representan vuelos, por lo tanto tiene una variedad de diferentes tipos de datos: número, texto, hora, etc.

Los datos representan vuelos comerciales en Estados Unidos durante el año 2021.

### Columnas

1. **Airline**: nombre de la aerolinea.
2. **CRSDepTime**: Hora de salida programada. (hhmm)
3. **DepTime**: Hora de salida verdadera. (hhmm)
4. **DepDelay**: Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas.
5. **CRSArrTime**: Hora de llegada programada. (hhmm)
6. **ArrTime**: Hora de llegada verdadera. (hhmm)
7. **ArrDelay**: Diferencia en minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas.
8. **ActualElapsedTime**: Tiempo de vuelo verdadero.
9. **CRSElapsedTime**: Tiempo de vuelo programado.
10. **Distance**: Distancia entre aeropuertos en millas.
11. **Year**: Año del vuelo.
12. **Month**: Mes del vuelo.
13. **DayofMonth**: Día del mes del vuelo.
14. **DayOfWeek**: Día de la semana del vuelo.
15. **Tail_Number**: Número de la cola, código para identificar la aeronave.
16. **Flight_Number_Operating_Airline**: Número del vuelo.
17. **Origin**: Aeropuerto de origen.
18. **OriginAirportID**: Código del aeropuerto de origen.
19. **OriginAirportSeqID**: Código del aeropuerto de origen.
20. **OriginCityName**: Nombre de la ciudad del aeropuerto de origen.
21. **OriginStateName**: Nombre de estado del aeropuerto de origen.
22. **Dest**: Aeropuerto de destino.
23. **DestAirportID**: Código del aeropuerto de destino.
24. **DestAirportSeqID**: Código del aeropuerto de destino.
25. **DestCityName**: Nombre de la ciudad del aeropuerto de destino.
26. **DestStateName**: Nombre de estado del aeropuerto de destino.
27. **Cancelled**: Si el vuelo fue cancelado, 1 = Sí, 0 = No.
28. **Diverted**: Si el vuelo fue desviado, 1 = Sí, 0 = No.

## Cargar datos

In [24]:
import os
import zipfile
import pathlib

direccion = pathlib.Path.cwd()

path_datos_zip = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\data.zip')

path_datos_csv = pathlib.Path.joinpath(direccion.parent.parent, 'Datos\\Combined_Flights_2021.csv')


if not os.path.isfile(path_datos_csv):
    try:
        with zipfile.ZipFile(path_datos_zip, mode="r") as archivo_zip:
            archivo_zip.extract('Combined_Flights_2021.csv', path=pathlib.Path.joinpath(direccion.parent.parent,'Datos'))
    except:
        print(".zip no existe")
    


In [48]:
df: DataFrame = spark.read.csv(path_datos_csv.absolute().as_posix(), header=True, inferSchema=True)

In [49]:
df.printSchema()

root
 |-- FlightDate: date (nullable = true)
 |-- Airline: string (nullable = true)
 |-- Origin: string (nullable = true)
 |-- Dest: string (nullable = true)
 |-- Cancelled: boolean (nullable = true)
 |-- Diverted: boolean (nullable = true)
 |-- CRSDepTime: integer (nullable = true)
 |-- DepTime: double (nullable = true)
 |-- DepDelayMinutes: double (nullable = true)
 |-- DepDelay: double (nullable = true)
 |-- ArrTime: double (nullable = true)
 |-- ArrDelayMinutes: double (nullable = true)
 |-- AirTime: double (nullable = true)
 |-- CRSElapsedTime: double (nullable = true)
 |-- ActualElapsedTime: double (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- DayofMonth: integer (nullable = true)
 |-- DayOfWeek: integer (nullable = true)
 |-- Marketing_Airline_Network: string (nullable = true)
 |-- Operated_or_Branded_Code_Share_Partners: string (nullable = true)
 |

In [50]:
#comprobacion de carga correcta con PySpark
df.describe().show()

+-------+--------------------+-------+-------+------------------+------------------+------------------+-----------------+------------------+-----------------+------------------+-----------------+-----------------+-----------------+-------+------------------+------------------+------------------+------------------+-------------------------+---------------------------------------+------------------------+---------------------------+-------------------------------+-----------------+------------------------+---------------------------+-----------+-------------------------------+------------------+------------------+------------------+--------------+-----------+------------------+---------------+------------------+------------------+-----------------+------------------+------------+---------+------------------+-------------+------------------+------------------+--------------------+----------+-----------------+------------------+------------------+-----------------+------------------+------

In [51]:
print((df.count(), len(df.columns)))

(6311871, 61)


In [52]:
#limitar a columnas más relevantes
columnas = [
    'Airline', # Aerolinea
    'CRSDepTime', #Hora de salida programada
    'DepTime', #Hora de salida actual
    'DepDelay', #Diferencia en minutos entre CRSDepTime y DepTime, tiempos negativos significa salidas tempranas
    'CRSArrTime',
    'ArrTime', #Hora de llegada
    'ArrDelay', #Diferencia de minutos entre la llegada programada y la hora de llegada, tiempos negativos significa llegadas tempranas
    'ActualElapsedTime', #Tiempo de vuelo verdadero
    'CRSElapsedTime', #Tiempo de vuelo programado
    'Distance', #Distancia entre aeropuertos en millas
    'Year', #Año del vuelo
    'Month', #Mes del vuelo
    'DayofMonth', #Dia del mes del vuelo
    'DayOfWeek', #Dia de la semana del vuelo
    'Tail_Number', #Numero de la cola, código para identificar la aeronave
    'Flight_Number_Operating_Airline', #Numero del vuelo
    
    'Origin', #Aeropuerto de origen
    'OriginAirportID', 'OriginAirportSeqID', #codigos del aeropuerto de origen
    'OriginCityName', #Nombre de la ciudad del aeropuerto de origen
    'OriginStateName', #Nombre de estado del aeropuerto de origen
    
    'Dest', #Aeropuerto de destino
    'DestAirportID', 'DestAirportSeqID', #codigos del aeropuerto de destino
    'DestCityName', #Nombre de la ciudad del aeropuerto de destino
    'DestStateName', #Nombre de estado del aeropuerto de destino
    
    'Cancelled', #El vuelo fue cancelado, 1 = Sí
    
    'Diverted', #Si el vuelo fue desviado, 1 = Sí
]

In [53]:
df: DataFrame = df.select(columnas)

In [54]:
print((df.count(), len(df.columns)))

(6311871, 28)
